Implement a 5-gram character language model from the English training data

In [39]:
from os import listdir,getcwd
from os.path import join, isfile
import collections
train4GramDict = collections.Counter()
train5GramDict = collections.Counter()
unigramDict = collections.Counter()
filepath = join(getcwd(), "gutenberg")
allFiles = [f for f in listdir(filepath) if not (f.startswith('.')) and isfile(join(filepath, f))]
for file in allFiles: 
    trainText = ''
    filename = join(filepath, file)
    for line in open(filename,'r+', encoding="latin1"):
    #         removing empty line
        if line != '\n':
            newLine = ''
            for ch in range(0,len(line)):
    # continuos space would get ignore
                if line[ch] == '\t':
                    newLine+=' '
                if ch != len(line)-1 and line[ch] == ' ' and line[ch] == line[ch+1]:
                    continue;
    #             in case of \n , replace it with space
                elif line[ch] == '\n':
                      newLine+=' '
    #             add any other character
                else:
                    newLine+=line[ch]
            trainText += newLine
    
    for charPos in range(0,len(trainText)):
        unigramDict[trainText[charPos]] +=1
    for charPos in range(0,len(trainText)-4):
        key = trainText[charPos:charPos+4]
        train4GramDict[key] +=1
    for charPos in range(0,len(trainText)-5):
        key = trainText[charPos:charPos+5]
        train5GramDict[key] +=1

In [54]:
# report the 4-gram and 5-gram character counts
print(sum(train5GramDict.values()))
print(sum(train4GramDict.values()))

11651645
11651664


In [41]:
threshold = 5
keysToDelete4Gram = []
train4GramDict['UNK'] = 0
for key in train4GramDict.keys():
    if key is not 'UNK' and train4GramDict[key] < threshold:
        train4GramDict['UNK'] += train4GramDict[key]
        keysToDelete4Gram.append(key)
for key in keysToDelete4Gram:
    del train4GramDict[key]

In [42]:
keysToDelete5Gram = []
train5GramDict['UNK'] = 0
for key in train5GramDict.keys():
    if key is not 'UNK' and train5GramDict[key] < threshold:
        train5GramDict['UNK'] += train5GramDict[key]
        keysToDelete5Gram.append(key)
for key in keysToDelete5Gram:
    del train5GramDict[key]

3.2 Report the average perplexity(per character) for file '03302_02.txt'. Use add-lambda smoothing (with lambda = 0.1)

In [44]:
#  processing the test file
from os import listdir,getcwd
from os.path import join
testText = ''
testFilepath = join(getcwd(), "SOC")
testFilename = join(testFilepath, '03302_02.txt')
for line in open(testFilename,'r+', encoding="latin1"):
#  removing empty line
    if line != '\n':
        newLine = ''
        for ch in range(0,len(line)):
# continuos space would get ignore
            if line[ch] == '\t':
                newLine+=' '
            if ch != len(line)-1 and line[ch] == ' ' and line[ch] == line[ch+1]:
                continue;
#             in case of \n , replace it with space
            elif line[ch] == '\n':
                  newLine+=' '
#             add any other character
            else:
                newLine+=line[ch]

        testText += newLine

In [45]:
len(testText)

6205262

In [46]:
test5Grams = []
for charPos in range(0,len(testText)-5):
    test5Grams.append(testText[charPos:charPos+5])

In [47]:
len(test5Grams)

6205257

In [48]:
from math import log, exp
modelLogProb = 0
K = 0.1
vocabSize = len(unigramDict)
N=len(test5Grams)
for gram5 in test5Grams:
    val5 = train5GramDict.get(gram5,None)
    gram4 = ''
    for ch in range(0,len(gram5)-1):
        gram4 += gram5[ch]
    val4 = train4GramDict.get(gram4,None)
    if val5 is None:
        val5 = train5GramDict['UNK']
    if val4 is None:
        val4 = train4GramDict['UNK']
    logProb = log((val5 + K)/(val4 + (K * vocabSize)))
    modelLogProb += logProb
perplexity = exp(modelLogProb * (-1/N))

In [49]:
perplexity

1.6028679391499376

In [50]:
modelLogProb

-2927606.040980314

3.3 report the names and scores of the three files with the highest perplexity

In [51]:
perplexityDict = {}
#  processing the test file
from os import listdir,getcwd
from os.path import join
from math import expm1,log, exp
testFilepath = join(getcwd(), "SOC")
allFiles = [f for f in listdir(testFilepath) if not (f.startswith('.')) and isfile(join(testFilepath, f))]
for file in allFiles:
    testFilename = join(testFilepath, file)
    testText = ''
    for line in open(testFilename,'r+', encoding="latin1"):
    #  removing empty line
        if line != '\n':
            newLine = ''
            for ch in range(0,len(line)):
    # continuos space would get ignore
                if line[ch] == '\t':
                    newLine+=' '
                if ch != len(line)-1 and line[ch] == ' ' and line[ch] == line[ch+1]:
                    continue;
    #             in case of \n , replace it with space
                elif line[ch] == '\n':
                      newLine+=' '
    #             add any other character
                else:
                    newLine+=line[ch]

            testText += newLine
#     computing 5 grams of text
    test5Grams = []
    for charPos in range(0,len(testText)-5):
        test5Grams.append(testText[charPos:charPos+5])
#     computing file perplexity
    fileLogProb = 0
    K = 0.1
    vocabSize = len(unigramDict)
    N=len(test5Grams)
    for gram5 in test5Grams:
        val5 = train5GramDict.get(gram5,None)
        gram4 = ''
        for ch in range(0,len(gram5)-1):
            gram4 += gram5[ch]
        val4 = train4GramDict.get(gram4,None)
        if val5 is None:
            val5 = train5GramDict['UNK']
        if val4 is None:
            val4 = train4GramDict['UNK']
        logProb = log((val5 + K)/(val4 + (K * vocabSize)))
        fileLogProb += logProb
    perplexity = exp(fileLogProb * (-1/N))
    perplexityDict[file] = perplexity

In [52]:
i=0
for key, value in sorted(perplexityDict.items(), key=lambda x:x[1], reverse=True):
    if i<3:
        i+=1
        print(key, value)

00101_01.txt 1.9651509541498489
00503_01.txt 1.8550437262621398
02803_02.txt 1.8076324619559627
